# Imports 

In [116]:
!pip install ccxt pandas numpy

In [117]:
!pip install ta-lib  

In [118]:
!pip install ta-lib talib-binary 
!pip install pandas-ta    

ERROR: Could not find a version that satisfies the requirement talib-binary (from versions: none)
ERROR: No matching distribution found for talib-binary


In [119]:
!pip uninstall logging

In [120]:
!pip install asyncio websocket-client
# !pip install sqlite3 
# !pip install logging  

In [121]:
!pip list

Package                   Version
------------------------- -----------
aiodns                    3.5.0
aiohappyeyeballs          2.6.1
aiohttp                   3.12.15
aiosignal                 1.4.0
altair                    5.5.0
asttokens                 3.0.0
async-timeout             5.0.1
asyncio                   4.0.0
attrs                     25.3.0
backcall                  0.2.0
blinker                   1.9.0
build                     1.3.0
cachetools                6.2.0
ccxt                      4.5.2
certifi                   2025.8.3
cffi                      1.17.1
charset-normalizer        3.4.3
click                     8.1.8
colorama                  0.4.6
comm                      0.2.3
contourpy                 1.3.0
cryptography              45.0.6
cycler                    0.12.1
dash                      3.2.0
debugpy                   1.8.16
decorator                 5.2.1
executing                 2.2.0
Flask                     3.1.2
fonttools             

In [122]:
!pip install plotly dash          
!pip install matplotlib seaborn   
!pip install streamlit 

In [123]:
!pip install python-dotenv       
!pip install schedule             
!pip install requests aiohttp     

^C


In [ ]:
!pip install scikit-learn

# Get Data

In [1]:
import os 
from dotenv import load_dotenv

In [2]:
load_dotenv()

False

In [3]:
class Configur():
    EXCHANGE  = "binance"
    SYMBOL = "BTC/USDT"
    TIMEFRAME = "1h"


    API_KEY = ""
    SECRET_KEY = ""
    SANDBOX = ""

    INIT_BALANCE = 10000
    RISK = 0.01
    STOP_LOSS = 0.02
    PROFIT_TAKE = 0.04
    # Strategy Parameters
    RSI_PERIOD = 14
    RSI_OVERSOLD = 30
    RSI_OVERBOUGHT = 70
    EMA_SHORT = 20
    EMA_LONG = 50
    
    # Execution Settings
    MIN_TRADES_FOR_METRICS = 50
    UPDATE_INTERVAL = 60  # seconds
    
    # Logging
    LOG_FILE = 'trades.csv'
    DB_FILE = 'trading_bot.db'



In [1]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

In [2]:
import ccxt
import pandas as pd
import numpy as np
import asyncio
import time
from datetime import datetime
from Config import Configs

class Manage_data:
    def __init__(self):
        self.exchange = self._init_exchange()
        self.current_price = 0
        self.ohlcv_data = pd.DataFrame()
        
    def _init_exchange(self):
        exchange_class = getattr(ccxt, 'binance')
        exchange = exchange_class({
            'apiKey': Configs.API_KEY,
            'secret': Configs.SECRET_KEY,
            'sandbox': Configs.SANDBOX,
            'enableRateLimit': True,
        })
        return exchange      
    
    def fetch_ohlcv(self, limit =800):
        try:
            ohlcv = self.exchange.fetch_ohlcv(
                "BTC/USDT",
                "1h",
                limit =limit
            )
            df = pd.DataFrame(ohlcv,columns=['timestamp','open','high','low','close','volume'])
            df.set_index('timestamp', inplace=True)
            self.ohlcv_data = df
            self.current_price = df['close'].iloc[-1]
            return df
        except Exception as e:
            print(e)
            return None
    def get_current_price(self):
        try :
            ticker = self.exchange.fetch_ticker("BTC/USDT")
            self.current_price = ticker['last']
            return self.current_price
        except Exception as e:
            print(f" problem in get_current_price {e}")
            return self.current_price
    def get_balance(self):
        try:
            balance = self.exchange.fetch_balance()
            return  balance['USDT']['free'] if 'USDT' in balance else Configur.INIT_BALANCE
        except Exception as e:
            print( f" problem in get_balance {e}")
            return Configur.INIT_BALANCE

# Make Dataframe

In [3]:
data = Manage_data()
df = data.fetch_ohlcv()
print(df)

                    open       high        low      close    volume
timestamp                                                          
1753966800000  118626.18  118692.08  117711.32  118573.28  13.16674
1753970400000  118568.63  119071.21  117273.45  117913.19  13.17513
1753974000000  117913.18  121615.28   30000.00  118297.31  30.31667
1753977600000  118297.31  143001.59   31748.44  118537.52  15.24438
1753981200000  118538.52  118737.14   95228.44  117733.18  14.95130
...                  ...        ...        ...        ...       ...
1756828800000  110829.14  132536.10  110473.69  110772.00  10.99981
1756832400000  110771.99  111050.00  109931.30  110914.01  21.87393
1756836000000  110914.00  110914.01  108592.00  110640.01  19.75668
1756839600000  110651.48  110829.15  110453.99  110794.84   9.91166
1756843200000  110794.84  111412.72  110729.45  111391.33  15.59797

[800 rows x 5 columns]


In [4]:
import numpy as np 
import pandas as pd 
import talib


def fet_Ext(df):
    df['price_change'] =  df['close'].pct_change()#15 min change 
    df['price_change_5'] =  df['close'].pct_change(5)#change in 75 mins
    price_range=(df['high'].rolling(14).max()-df['low'].rolling(14).min())
    df['pos_range_prc'] = df['close']-df['low'].rolling(14).min()/price_range
    
    df['rsi'] = talib.RSI(df['close'].values,14)
    df['rsi_oversold'] = (df['rsi'] < 30).astype(int)# stock sold too much will rise
    df['rsi_overbought'] = (df['rsi'] > 80).astype(int)# stock bought too much will fall

    #moving AVGS
    df['smal_10'] = df['close'].rolling(10).mean()# Short term mean
    df['smal_20'] = df['close'].rolling(20).mean()# Long term mean
    df['price_above_smal_10'] = (df['close']>df['smal_10']).astype(int)
    df['price_above_smal_20'] = (df['close']>df['smal_20']).astype(int)
    df['sma_trend'] = (df['smal_10'] > df['smal_20']).astype(int)

    
    df['volume_avg'] = df['volume'].rolling(20).mean()
    df['volume_high'] = (df['volume'] > df['volume_avg'] * 1.5).astype(int)



    df['volatility'] = df['price_change'].rolling(10).std()
    df['high_volatility'] = (df['volatility'] > df['volatility'].rolling(50).mean()).astype(int)

    df['future_return'] = df['price_change'].shift(-1)
    df['target'] = np.where(df['future_return'] > 0.002, 1, 
                            np.where(df['future_return'] < -0.002, -1, 0))
    
    return df.fillna(0)

In [5]:
df = fet_Ext(df)

In [6]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1753966800000,118626.18,118692.08,117711.32,118573.28,13.16674,0.000000,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.005567,-1
1753970400000,118568.63,119071.21,117273.45,117913.19,13.17513,-0.005567,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.003258,1
1753974000000,117913.18,121615.28,30000.00,118297.31,30.31667,0.003258,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.002031,1
1753977600000,118297.31,143001.59,31748.44,118537.52,15.24438,0.002031,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.006786,-1
1753981200000,118538.52,118737.14,95228.44,117733.18,14.95130,-0.006786,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.000641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756828800000,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0
1756832400000,110771.99,111050.00,109931.30,110914.01,21.87393,0.001282,0.019468,110913.833529,58.872665,0,...,110113.4330,1,1,1,23.176437,0,0.008059,1,-0.002470,-1
1756836000000,110914.00,110914.01,108592.00,110640.01,19.75668,-0.002470,-0.004443,110639.833529,56.016149,0,...,110223.1330,1,1,1,23.690536,0,0.008116,1,0.001399,0


In [10]:
feat_df = fet_Ext(df)

In [11]:
len(df)

800

# Train Model -> Random Forest

In [13]:
from Config import Configur

ImportError: cannot import name 'Configur' from 'Config' (c:\Users\Com\OneDrive\Documents\GitHub\Crypto_bot\Config.py)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import pickle


MIN_DATA_POINTS = 300
class RandomForestStrat:
    def __init__(self):
        self.name = "Random_forest_classifier"
        self.model = RandomForestClassifier(
            n_estimators = 50,
            max_depth=8,
            random_state= 42,
            class_weight='balanced'
        )
        self.features = fet_Ext
        self.feature_list = [
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.is_trained = False


    def Train(self,df):
        df = fet_Ext(df)
        feature_columns =  self.feature_list+ ['target']
        data = df[feature_columns].dropna()
        if len(data) < MIN_DATA_POINTS:
            #More data points are required
            return False 
        X = data[self.feature_list]
        Y = data['target']

        self.model.fit(X,Y)

        y_pred = self.model.predict(X)
        accuracy = accuracy_score(Y,y_pred)

        print(f"Accuracy:{accuracy}")
        with open("Weighted_model.pkl", "wb") as f:
            pickle.dump(self.model, f)

        self.is_trained = True
        return True
    
    def predict(self, df):
        """Make prediction"""
        if not self.is_trained:
            return 'hold', 0.0
        
        # Create features for latest data
        df = self.features(df)
        latest_data = df[self.feature_list].iloc[-1:].fillna(0)
        
        # Get prediction and confidence
        prediction = self.model.predict(latest_data)[0]
        probabilities = self.model.predict_proba(latest_data)[0]
        confidence = max(probabilities)
        
        # Convert to signal
        if confidence > 0.8:
            if prediction == 1:
                return 'buy', confidence
            elif prediction == -1:
                return 'sell', confidence
        
        return 'hold', confidence

In [14]:
mod = RandomForestStrat()

In [15]:
model  = mod.Train(df)

Accuracy:0.885


# Test It out

In [7]:
df[:1]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1753966800000,118626.18,118692.08,117711.32,118573.28,13.16674,0.0,0.0,0.0,0.0,0,...,0.0,0,0,0,0.0,0,0.0,0,-0.005567,-1


In [8]:
df.tail()

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756828800000,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0
1756832400000,110771.99,111050.00,109931.30,110914.01,21.87393,0.001282,0.019468,110913.833529,58.872665,0,...,110113.4330,1,1,1,23.176437,0,0.008059,1,-0.002470,-1
1756836000000,110914.00,110914.01,108592.00,110640.01,19.75668,-0.002470,-0.004443,110639.833529,56.016149,0,...,110223.1330,1,1,1,23.690536,0,0.008116,1,0.001399,0
1756839600000,110651.48,110829.15,110453.99,110794.84,9.91166,0.001399,-0.005146,110794.663529,57.277593,0,...,110301.0040,1,1,1,23.779571,0,0.008123,1,0.005384,1
1756843200000,110794.84,111412.72,110729.45,111391.33,15.59797,0.005384,0.005073,111391.153529,61.820536,0,...,110419.5205,1,1,1,23.841936,0,0.008234,1,0.000000,0


In [9]:
df[-1:]

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
1756843200000,110794.84,111412.72,110729.45,111391.33,15.59797,0.005384,0.005073,111391.153529,61.820536,0,...,110419.5205,1,1,1,23.841936,0,0.008234,1,0.0,0


In [10]:
import pickle

with open("Weighted_model.pkl", "rb") as f:
    models = pickle.load(f)
test_df = df[-100:]
test_df = test_df.drop(['target'], axis=1)
feature_list = [
'price_change', 'price_change_5', 'pos_range_prc',
'rsi', 'rsi_oversold', 'rsi_overbought',
'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
'volume_high', 'high_volatility']
X = test_df[feature_list]
dictions  = models.predict(X)

In [11]:
from sklearn.metrics import accuracy_score
res_df = pd.DataFrame()
res_df['results'] = dictions
accuracy = accuracy_score(res_df['results'],df['target'][-100:])
print(accuracy)

0.87


In [12]:
df['target'].value_counts()

target
 0    448
 1    181
-1    171
Name: count, dtype: int64

Model result comes in -1 -sell , 0 -hold ,1 buy

In [13]:
dictions[-20:]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  1,  1, -1,  0,  0,  0,
        0,  0,  1])

In [14]:
side = {-1:"sell",0:"hold",1:"buy"}

# Prepping DF for model 

In [15]:
df.index

Index([1753966800000, 1753970400000, 1753974000000, 1753977600000,
       1753981200000, 1753984800000, 1753988400000, 1753992000000,
       1753995600000, 1753999200000,
       ...
       1756810800000, 1756814400000, 1756818000000, 1756821600000,
       1756825200000, 1756828800000, 1756832400000, 1756836000000,
       1756839600000, 1756843200000],
      dtype='int64', name='timestamp', length=800)

In [16]:
df.index = pd.to_datetime(df.index, unit="ms")

In [17]:
df

,open,high,low,close,volume,price_change,price_change_5,pos_range_prc,rsi,rsi_oversold,...,smal_20,price_above_smal_10,price_above_smal_20,sma_trend,volume_avg,volume_high,volatility,high_volatility,future_return,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-07-31 13:00:00,118626.18,118692.08,117711.32,118573.28,13.16674,0.000000,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.005567,-1
2025-07-31 14:00:00,118568.63,119071.21,117273.45,117913.19,13.17513,-0.005567,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.003258,1
2025-07-31 15:00:00,117913.18,121615.28,30000.00,118297.31,30.31667,0.003258,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,0.002031,1
2025-07-31 16:00:00,118297.31,143001.59,31748.44,118537.52,15.24438,0.002031,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.006786,-1
2025-07-31 17:00:00,118538.52,118737.14,95228.44,117733.18,14.95130,-0.006786,0.000000,0.000000,0.000000,0,...,0.0000,0,0,0,0.000000,0,0.000000,0,-0.000641,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-02 16:00:00,110829.14,132536.10,110473.69,110772.00,10.99981,-0.000516,0.009650,110771.823529,57.837930,0,...,109957.7730,1,1,1,22.534124,0,0.008076,1,0.001282,0
2025-09-02 17:00:00,110771.99,111050.00,109931.30,110914.01,21.87393,0.001282,0.019468,110913.833529,58.872665,0,...,110113.4330,1,1,1,23.176437,0,0.008059,1,-0.002470,-1
2025-09-02 18:00:00,110914.00,110914.01,108592.00,110640.01,19.75668,-0.002470,-0.004443,110639.833529,56.016149,0,...,110223.1330,1,1,1,23.690536,0,0.008116,1,0.001399,0


In [22]:
%load_ext autoreload
%autoreload 2

# Using Model in latest Data

In [ ]:
import asyncio
import pickle
from datetime import datetime
from GetData import Manage_data
from Manage_orders import ExecutionEngine
from Logger import TradeLogger
from Visualizer import Visualizerr
from Config import Configs
from FeatureExtract import Features
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
import threading


class CryptoTradingBot:
    def __init__(self, df=None):
        self.data_manager = Manage_data()
        self.execution_engine = ExecutionEngine()
        self.logger = TradeLogger()
        self.visualizer = Visualizerr()
        self.last_timestamp = None
        self.running = False
        self.Extractor = Features()
        self.feature_list  =[
        'price_change', 'price_change_5', 'pos_range_prc',
        'rsi', 'rsi_oversold', 'rsi_overbought',
        'price_above_smal_10', 'price_above_smal_20', 'sma_trend',
        'volume_high', 'high_volatility']
        self.position = None
        self.test_df = df  # premade DataFrame
        with open("Weighted_model.pkl", "rb") as f:
            self.models = pickle.load(f)

    def run_bot(self):
        """Main bot execution loop"""
        self.running = True
        
        print(f"Starting crypto trading bot for {Configs.SYMBOL}")
        print(f"Strategy: Random Forest ")
        print(f"Initial Balance: ${Configs.INIT_BALANCE}")
        print("-" * 50)
        

        for i in range( 100, len(self.test_df)-1):  # start after enough candles for indicators
            try:
                df = self.test_df.iloc[: i+1].copy()

                clean_df = df.dropna()
                Ext_Features = self.Extractor.fet_Ext(clean_df)
                feat_list = self.Extractor.feature_list
                clean_df = Ext_Features[feat_list]
                
                self.signal = self.models.predict(clean_df)
                
                self.side = {-1: "sell", 0: "hold", 1: "buy"}

                current_price = df["close"].iloc[-1]
                current_time = df.index[-1]
                # print(self.side[self.signal[-1]])

                # # Check exit conditions
                self.execution_engine.check_exit_conditions(current_price, current_time)

                # # Execute new signal
                self.execution_engine.execute_signal(self.side[self.signal[-1]], current_price, current_time)

                # Log completed trades
                if self.execution_engine.trades:
                    for trade in self.execution_engine.trades:
                        if trade not in getattr(self, 'logged_trades', []):
                            self.logger.log_trade(trade)
                            if not hasattr(self, 'logged_trades'):
                                self.logged_trades = []
                            self.logged_trades.append(trade)

                # Display status
                self.display_status(df, self.signal, current_price)

            except Exception as e:
                print(f"Error in backtest loop: {e}")
                break
    def display_status(self, df, signal, price):
        """Display current bot status"""
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        rsi = df['rsi'].iloc[-1] if 'rsi' in df.columns else 0
        
        print(f"\n[{current_time}] Price: ${price:.2f} | RSI: {rsi:.1f} | Signal: {signal}")
        
        if self.execution_engine.position:
            pos = self.execution_engine.position
            unrealized_pnl = pos.calculate_pnl(price)
            print(f"Position: {pos.side.upper()} | Size: {pos.size:.4f} | Entry: ${pos.entry_price:.2f} | Unrealized P&L: ${unrealized_pnl:.2f}")
        
        print(f"Balance: ${self.execution_engine.balance:.2f} | Total Trades: {len(self.execution_engine.trades)}")
    def start_dashboard(self):
        """Start web dashboard in separate thread"""
        bot_ref =  self
        def run_dash():
            # Setup callbacks for live updates
            
            @self.visualizer.app.callback(
                [Output('price-chart', 'figure'),
                 Output('equity-chart', 'figure'),
                 Output('metrics-display', 'children')],
                [Input('interval-component', 'n_intervals')]
            )
            def update_dashboard(n_intervals):
                # Get latest data

                print(f"Dashboard updating... iteration {n_intervals}")  # Debug
                
                df = bot_ref.current_df.copy() if hasattr(bot_ref, 'current_df') and bot_ref.current_df is not None else bot_ref.test_df.copy()
                df = bot_ref.Extractor.fet_Ext(df)
                
                price_fig = self.visualizer.create_price_chart(
                    df, 
                    self.execution_engine.trades, 
                    df['close'].iloc[-1] if len(df) > 0 else 0
                )
                
                equity_fig = self.visualizer.create_equity_curve(
                    self.execution_engine.equity_curve
                )
                
                # Create metrics display
                if len(self.execution_engine.trades) > 0:
                    metrics = self.logger.calculate_metrics(self.execution_engine.trades)
                    metrics_div = html.Div([
                        html.H3("Performance Metrics"),
                        html.P(f"Win Rate: {metrics['win_rate']:.1f}%"),
                        html.P(f"Total P&L: ${metrics['total_pnl']:.2f}"),
                        html.P(f"Total Trades: {metrics['total_trades']}"),
                        html.P(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}"),
                        html.P(f"Max Drawdown: ${metrics['max_drawdown']:.2f}")
                    ])
                else:
                    metrics_div = html.Div([html.H3("Waiting for trades...")])
                
                return price_fig, equity_fig, metrics_div
            
            self.visualizer.app.run(debug=False, host='127.0.0.1', port=8050)
        
        dashboard_thread = threading.Thread(target=run_dash)
        dashboard_thread.daemon = True
        dashboard_thread.start()
        print("Dashboard started at http://127.0.0.1:8050")

In [ ]:
# Load or create your test DataFrame
# df = pd.read_csv("your_test_data.csv", index_col=0, parse_dates=True)

# Pass it into the bot
bot = CryptoTradingBot(df=df)
# Start dashboard
bot.start_dashboard()

# Wait a moment for dashboard to start
time.sleep(2)

# Run trading bot
bot.run_bot()

Dashboard started at http://127.0.0.1:8050


Dashboard updating... iteration 0
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')
Shape: (800, 22)
                          open       high        low      close    volume  \
timestamp                                                                   
2025-07-31 13:00:00  118626.18  118692.08  117711.32  118573.28  13.16674   
2025-07-31 14:00:00  118568.63  119071.21  117273.45  117913.19  13.17513   
2025-07-31 15:00:00  117913.18  121615.28   30000.00  118297.31  30.31667   
2025-07-31 16:00:00  118297.31  143001.59   31748.44  118537.52  15.24438   
2025-07-31 17:00:00  118538.52  118737.14   95228.44  117733.18  14.95130   
...                  

Dashboard updating... iteration 5
Visualizer got df columns: Index(['open', 'high', 'low', 'close', 'volume', 'price_change',
       'price_change_5', 'pos_range_prc', 'rsi', 'rsi_oversold',
       'rsi_overbought', 'smal_10', 'smal_20', 'price_above_smal_10',
       'price_above_smal_20', 'sma_trend', 'volume_avg', 'volume_high',
       'volatility', 'high_volatility', 'future_return', 'target'],
      dtype='object')
Shape: (800, 22)
                          open       high        low      close    volume  \
timestamp                                                                   
2025-07-31 13:00:00  118626.18  118692.08  117711.32  118573.28  13.16674   
2025-07-31 14:00:00  118568.63  119071.21  117273.45  117913.19  13.17513   
2025-07-31 15:00:00  117913.18  121615.28   30000.00  118297.31  30.31667   
2025-07-31 16:00:00  118297.31  143001.59   31748.44  118537.52  15.24438   
2025-07-31 17:00:00  118538.52  118737.14   95228.44  117733.18  14.95130   
...                  